In [99]:
import json
with open('dianping.json') as f:
    d = json.load(f)

# 5986 listings total
# breadcrumb: [{<class 'str'>}, 5986]
# fuwu: [{<class 'int'>, <class 'float'>}, 5985]
# city: [{<class 'str'>}, 5986]
# reviews: [{<class 'list'>}, 5986]
# name: [{<class 'str'>}, 5986]
# branch: [{<class 'str'>, <class 'NoneType'>}, 5986]
# full_name: [{<class 'str'>}, 5986]
# category: [{<class 'str'>, <class 'NoneType'>}, 5986]
# distict: [{<class 'str'>}, 5986]
# kouwei: [{<class 'int'>, <class 'float'>}, 5985]
# cost_per: [{<class 'str'>}, 5986]
# star_rank: [{<class 'int'>, <class 'float'>}, 5986]
# huanjing: [{<class 'int'>, <class 'float'>}, 5985]
# address: [{<class 'str'>}, 5986]

# review_id: [{<class 'str'>}, 346381]
# fuwu: [{<class 'int'>, <class 'NoneType'>}, 346381]
# huanjing: [{<class 'int'>, <class 'NoneType'>}, 346381]
# text: [{<class 'str'>}, 346381]
# text-seg: [{<class 'str'>}, 346381]
# star_rank: [{<class 'int'>}, 346381]
# member_name: [{<class 'str'>}, 346381]
# annotations: [{<class 'list'>}, 631] <-- skipping: sparse + random junk 
# kouwei: [{<class 'int'>, <class 'NoneType'>}, 346381]
# member_id: [{<class 'str'>}, 346381]
# date_posted: [{<class 'str'>}, 346381]
# comm_type: [{<class 'str'>, <class 'NoneType'>}, 346381]
# cost_per: [{<class 'str'>, <class 'NoneType'>}, 346381]

In [105]:
from collections import OrderedDict
parsed = []
for listing, stuff in d.items():
    if not stuff.get('fuwu'): # there is one listing that is missing fuwu/kouwei/huanjing 
        continue
    curr = OrderedDict()
    curr['restaurant_id'] = listing
    curr['restaurant_name'] = stuff['name']
    curr['restaurant_full_name'] = stuff['full_name']
    curr['breadcrumb'] = stuff['breadcrumb']
    curr['city'] = stuff['city']
    curr['district'] = stuff['distict']
    curr['address'] = stuff['address']
    curr['branch'] = stuff['branch']
    curr['category'] = stuff['category']
    curr['service_agg'] = float(stuff['fuwu'])
    curr['rank_agg'] = float(stuff['star_rank'])
    curr['ambience_agg'] = float(stuff['huanjing'])
    curr['taste_agg'] = float(stuff['kouwei'])
    if len(stuff['cost_per']) > 1: # listing has cost_per = "-"
        curr['cost_agg'] = float(stuff['cost_per'][:-1]) # delete unit yuan
    for review in stuff['reviews']:
        data = OrderedDict()
        data['text_seg'] = review['text-seg']
        if not data['text_seg']:
            continue
        data['text'] = review['text']
        data['review_id'] = review['review_id'] 
        data['date_raw'] = review['date_posted']
        date = data['date_raw'].split("-") 
        month = 0
        if len(date) == 2: # MM-DD
            month = date[0]
        elif len(date) == 3: #YY-MM-DD
            month = date[1]
        else:
            # 03-23  更新于15-03-23 22:40 [u'03', u'23\xa0\xa0\u66f4\u65b0\u4e8e15', u'03', u'23 22:40']
            # 14-11-23  更新于11-23 11:39 [u'14', u'11', u'23\xa0\xa0\u66f4\u65b0\u4e8e11', u'23 11:39']
            # 14-08-31  更新于15-02-15 19:33 [u'14', u'08', u'31\xa0\xa0\u66f4\u65b0\u4e8e15', u'02', u'15 19:33']
            if len(date) in [4, 5]:
                if len(date[1]) == 2:
                    month = date[1]
                else:
                    month = date[0]
            else:
                print("shouldn't hit here, {}".format(data['date_raw']))
        data['month'] = int(month) # zero denotes no month found
        data['season'] = ''
        if data['month'] in [3, 4, 5]:
            data['season'] = 'spring'
        if data['month'] in [6, 7, 8]:
            data['season'] = 'summer'
        if data['month'] in [9, 10, 11]:
            data['season'] = 'fall'
        if data['month'] in [12, 1, 2]:
            data['season'] = 'winter'
        data['member_id'] = review['member_id'][8:]
        data['member_name'] = review['member_name']
        data['review_type'] = review['comm_type']
        data['service'] = review['fuwu'] # already int (or None)
        data['rank'] = review['star_rank']
        data['ambience'] = review['huanjing']
        data['taste'] = review['kouwei']
        data['cost'] = review['cost_per']
        if data['cost']:
            data['cost'] = float(review['cost_per'][5:])
        parsed.append(OrderedDict(data.items() + curr.items()))
print("{} reviews found".format(len(parsed)))

346351 reviews found


In [107]:
import unicodecsv as csv
with open('reviews_formatted_no_header.tsv', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', encoding='utf-8')
    for i, record in enumerate(parsed):
#         if i == 0:
#             writer.writerow([k for k, v in record.items()]) 
        writer.writerow([v for k, v in record.items()]) 
tsvfile.close()
print("done writing")

done writing
